In [248]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from dataclasses import dataclass

In [249]:
#initial settings
PUZZLE_DIM = 3
#action = namedtuple('Action', ['pos1', 'pos2'])
SOLUTION = np.array([n+1 for n in range(PUZZLE_DIM*PUZZLE_DIM)])
SOLUTION[-1] = 0
puzzle = SOLUTION[:]
print(SOLUTION)

[1 2 3 4 5 6 7 8 0]


In [250]:
#utility functions and structures

#action class

@dataclass
class State:
    board: np.array
    gn: int = 0
    fn: int = 0
    
@dataclass
class Action:
    x1: int
    y1: int
    x2: int
    y2: int
    #gn: int = 0
    #fn: int = 0

def print_board(state: State):
    state = state.board
    matrix = np.resize(state, (PUZZLE_DIM, PUZZLE_DIM))
    print(matrix)

In [251]:
def available_actions(state: State) -> list['Action']:
    #find the empty tile
    board = state.board
    board = np.resize(board, (PUZZLE_DIM, PUZZLE_DIM))
    ii = np.where(board == 0)
    x= ii[0][0]
    y= ii[1][0]
    #print(x,y)
    #print_board (state)
    actions = list()
    if x > 0:
        actions.append(Action(x, y, x - 1, y))
    if x < PUZZLE_DIM - 1:
        actions.append(Action(x, y, x + 1, y))
    if y > 0:
        actions.append(Action(x, y, x, y - 1))
    if y < PUZZLE_DIM - 1:
        actions.append(Action(x, y, x, y + 1))
    return actions



def do_action(state: State, action: 'Action') -> State:
    new_board = state.board.copy()
    new_board[action.x1, action.y1], new_board[action.x2, action.y2] = new_board[action.x2, action.y2], new_board[action.x1, action.y1]
    new_state = State(new_board, state.gn + 1, 0)
    #print_board(new_state)
    return new_state

In [252]:
RANDOMIZE_STEPS = 100_000
state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)),0,0)
#print_board(state)
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
print_board(state)

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

[[5 1 6]
 [4 3 8]
 [0 2 7]]


In [253]:
def compute_hcost_manhattan(state: State) -> int:
    board = state.board
    board = np.resize(board, (PUZZLE_DIM, PUZZLE_DIM))
    distance = 0
    solution = np.resize(SOLUTION, (PUZZLE_DIM, PUZZLE_DIM))
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if board[i, j] == 0:
                continue
            ii = np.where(solution == board[i, j])
            x= ii[0][0]
            y= ii[1][0]
            x, y = divmod(board[i, j] - 1, PUZZLE_DIM)
            distance += abs(x - i) + abs(y - j)
    return distance

compute_hcost_manhattan(state)

np.int64(12)

In [ ]:
def A_star_search(initial_state: State):
    initial_state.fn = compute_hcost_manhattan(initial_state)
    frontier = [initial_state]
    visited = []
    total_cost=1
    i=0
    score = 0
    while frontier:
        current = min(frontier, key=lambda x: x.fn)
        if np.array_equal(current.board, SOLUTION):
            return current, total_cost
        
        frontier.remove(current)
        visited.append(current)
        for action in available_actions(current):
            new_state = do_action(current, action)
            new_state.fn = compute_hcost_manhattan(new_state)
       
        i+=1
        if i%300==0:
            print(i)
            print_board(current)
        return current, total_cost

solution, cost = A_star_search(state)
print_board(solution)

AttributeError: 'list' object has no attribute 'board'